# Hackathon

Some utilities

## Import Utils

In [12]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/full.h5'
OUT_PATH = '/notebooks/data/balanced_7classes.h5'

In [18]:
batch_size = 300000
nb_samples = 18698240
batch_count = int(nb_samples/batch_size) + 1

nb_per_class = dict()
nb_per_class[1] = 0
nb_per_class[2] = 0
nb_per_class[3] = 0
nb_per_class[5] = 0
nb_per_class[10] = 0
nb_per_class[12] = 0
nb_per_class[19] = 0

In [ ]:
import os

if os.path.exists(OUT_PATH):
        os.remove(OUT_PATH)
        
f = h5.File(OUT_PATH, 'w')
top_landcover_submit = f.create_dataset("TOP_LANDCOVER", [0, 1], maxshape=(None, 1))
s2_submit = f.create_dataset("S2", [0, 16,16,4], maxshape=(None, 16,16,4))

for b in range(batch_count):
    
    if (min(nb_per_class.values()) < batch_size) :

        print('b : ' + str(b))
        print(nb_per_class)
        top_landcover = h5.File(PATH_DATA)['TOP_LANDCOVER'][b*batch_size:(b+1)*batch_size -1]
        top_landcover = top_landcover[0:].astype(int)
        images = h5.File(PATH_DATA)['S2'][b*batch_size:(b+1)*batch_size -1, :, :, :]

        unique, counts = np.unique(top_landcover, return_counts=True)

        indexs = []

        for key in nb_per_class.keys():

            if(nb_per_class[key] < batch_size) and key in unique:

                print('key : ' + str(key))

                index_key = np.where(top_landcover == key)[0]

                if len(index_key) > (batch_size - nb_per_class[key]):
                    index_key = index_key[:batch_size - nb_per_class[key]]

                if len(indexs) == 0 :
                    indexs = index_key
                else :
                    indexs = np.append(indexs, index_key)
                nb_per_class[key] += len(index_key)

        if len(indexs) > 0 :
            indexs = np.sort(np.array(indexs)).astype(int)

            top_landcover_batch = top_landcover[indexs]
            images_batch = images[indexs, :, :, :]

            top_landcover_submit.resize(top_landcover_submit.shape[0] + len(top_landcover_batch), axis=0)
            top_landcover_submit[-len(top_landcover_batch):] = top_landcover_batch[:]
            
            s2_submit.resize(s2_submit.shape[0] + len(top_landcover_batch), axis=0)
            s2_submit[-len(top_landcover_batch):, ::] = images_batch
            
    else :
        break

f.close()  
                

b : 0
{19: 0, 1: 0, 2: 0, 3: 0, 5: 0, 10: 0, 12: 0}
key : 19
key : 1
key : 2
key : 3
key : 5
key : 10
key : 12
b : 1
{19: 4, 1: 34732, 2: 39381, 3: 18914, 5: 774, 10: 8, 12: 5273}
key : 19
key : 1
key : 2
key : 3
key : 5
key : 10
key : 12
b : 2
{19: 23, 1: 68850, 2: 120726, 3: 100421, 5: 1580, 10: 159, 12: 106115}
key : 19
key : 1
key : 2
key : 3
key : 5
key : 10
key : 12
b : 3
{19: 291, 1: 69888, 2: 130357, 3: 109796, 5: 169399, 10: 272, 12: 135544}
key : 1
key : 2
key : 3
key : 5
key : 10
key : 12
b : 4
{19: 291, 1: 70130, 2: 136720, 3: 117176, 5: 300000, 10: 489, 12: 158569}
key : 19
key : 1
key : 2
key : 3
key : 10
key : 12
b : 5
{19: 307, 1: 79757, 2: 158407, 3: 156248, 5: 300000, 10: 1316, 12: 189253}
key : 19
key : 1
key : 2
key : 3
key : 10
key : 12
b : 6
{19: 404, 1: 98553, 2: 203451, 3: 265869, 5: 300000, 10: 4855, 12: 254256}
key : 19
key : 1
key : 2
key : 3
key : 10
key : 12
b : 7
{19: 409, 1: 109616, 2: 235197, 3: 300000, 5: 300000, 10: 6639, 12: 300000}
key : 19
key : 1
k